# QISKit Hello World!

Jean-Michel Torres 2019-03-19, MESO@LR 

<br>
<img src="./images/BY-NC-SA.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="200 px" align="left">
<br>

# A. No QISKit : python, complexity

## Jupyter

In [ ]:
a = 1

In [ ]:
print(a)

## Python

### Import

In [ ]:
from math import sin, sqrt

a = 2
b = sqrt(a)
print(b)
c = sin(a)
print(c)

### Loop, test, and Monte-Carlo

In [ ]:
import random
import time

start_time = time.time()

#---> calculation starts here
iterations = 5_000_000                                 # change this for accuracy vs duration 
hit = 0                                                 # counter of random hits with module < 1 
for i in range(iterations):
    if random.random()**2 + random.random()**2 < 1:     # random(a,b) returns a pseudo random float between a and b
        hit += 1
#---> calulation ends here
     
    
end_time = time.time()
duration = end_time - start_time

print(f"Total execution time: {duration:.1f} seconds")
print(f"Pi = {4 * hit / iterations}")

### Plotting

In [ ]:
import matplotlib.pyplot as plt
import math

a = [x for x in range(100)]

b = [x**2 for x in range(100)]
c = [80 * x * math.cos(x / 5) for x in range(100)]
d = [1000 * math.log2(x+1) for x in range(100)]

plt.plot(a,b, color="blue")
plt.plot(a,c, color="red")
plt.plot(a,d, color="green")

plt.grid
plt.show


## Computing can become huge

### finding the factors of a large integer

In [ ]:
import time
from math import sqrt,floor

start_time = time.time()
# choose one (depending on your ratio patience/CPU)
d =         6721502531819 #(~0.4 sec)  (13)
#d =    40355369611024687 #(~30 sec)   (17)
#d = 70552560783426882343 #(~2100 sec) (20)

maxCalc = floor(sqrt(d))

for i in range(2,maxCalc):
    if d % i == 0:
        print(i)
        break

end_time = time.time()
duration = end_time - start_time
print(f"Total execution time: {duration:.1f} seconds")
print(f"Loops per second {i / duration:.0f}")

# B. QISKit, no Quantum, single qbit

#### imports
you need to import from qiskit : 
<ul>
    <li>QuantumRegister : to define and use qubits</li>
    <li>ClassicalRegister : to get a "classical" measurement of the qubits after computation has happened</li>
    <li>QuantumCircuit to be able to compose your calculation</li>
    <li>execute : this will be used to actually "run" your circuit</li>
</ul>    


In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute

#### define needed registers and build quantum circuit
for example let's define `qr` as a quantum register of size 1 (will use 1 qubit)

`<register_name> = QuantumRegister(size)` 

and let's define `cr` as a classical register with that same size. Siminal notation use ClassicalRegister constructor


In [ ]:
# deinfe registers here, size 1
cr = ClassicalRegister(1)
qr = QuantumRegister(1)

## <span style="color:red"><em>Example 1 : IDENTITY</em></span>

The simplest circuits of alL.


First, let's define a quantum circuit using `qc` and `cr`


`<circuit> = QuantumCircuit(<quantum_register>, <classical_register>)`


In [ ]:
# define quantum  circuit here 
qc_id = QuantumCircuit(qr,cr)

Then you can add a gate (in this case `iden` gate is identity: does nothing), this way: 

`<circuit>.gate(<qubits>)`

where `<circuit>` is the circuit object you have created above, and `<qubits>`is the list of qubit on which the gate applies `qr[0]` in this case.


In [ ]:
# add identity gate here 
qc_id.iden(qr[0])

Add another gate to your circuit, this one is special : it comes at the end and uses `qr` and `cr`

`<circuit>.measure(<qr,cr>)`    

this will measure the states of the qubits in the quantum register, and load values into the classical register.

In [ ]:
# add measurement gate here 
qc_id.measure(qr,cr)

You can now visualize the circuit using the .draw() method on your circuit : `<circuit>.draw()`
 

In [ ]:
# use draw method: 
qc_id.draw()

### prepare for execution

Here we select on which device actual machine or simulator we will run our experiment
Here we select the local simulator.

In [ ]:
from qiskit import Aer
print(Aer.backends())
backend = Aer.get_backend('qasm_simulator')

### execute, and read results

Now we execute our experiment using the `execute` function, returning a "job result" object:

`execute()` uses these parameters: 
<ul>
    <li>circuit object</li>
    <li>the device on which we execute (in our example this is stored in `backend`)</li>
    <li>the number of "shots" we want to run, eg: `shots=1024`</li>
</ul>  

It goes: 

`<myjob> = execute(<my_circuit>,<backend>,shots=nnnn)` 

Then we store the result in a variable `<my_result>` using `.result()` method on the job

Finally, we can print `<my_result>.get_counts(<my_circuit>)`


In [ ]:
# define job, get results
job = execute(qc_id,backend,shots=1024)
my_result = job.result()
print(my_result.get_counts(qc_id))  



In [ ]:
# change the value in get_count method (your circuit name)
from qiskit.tools.visualization import plot_histogram
plot_histogram(my_result.get_counts(qc_id))

## <span style="color:red"><em>Example 2: NOT (bit flip)</em></span>

Define a new quantum circuit, and use the bit flip gate: `<quantum_circuit>.x(<qubit>)`, use mearure gate, you can draw the circuit, execute the epxeriment and read the result as above. You may add Juptyer cells as needed to progress step by step. 

In [ ]:
#define quantum circuit
qc_not = QuantumCircuit(qr,cr)

In [ ]:
# add x gate on qubit 0

qc_not.x(qr[0])

In [ ]:
# add measurement gate from quantum register to classical register
qc_not.measure(qr,cr)

In [ ]:
# draw you quantum circuit
qc_not.draw()

In [ ]:
# execute and get results 
job = execute(qc_not,backend, shots=1024)
result = job.result()
result.get_counts(qc_not)

In [ ]:
# change the value in get_count method (your circuit name)
from qiskit.tools.visualization import plot_histogram
plot_histogram(result.get_counts(qc_not))

# C. QISKit, no Quantum, multi-qbit gates

## <span style="color:red"><em>Example 3: CNOT controlled not </em></span>

flips state of qbit b if qbit a is 1 (else leave b in original state).

The cnot gate is : `cx(<ctrl_qubit,target_qubit)`

Don't forget to define quantum and classical registers with size 2. 
The build a quantum circuit. You can use `x` gate on qbit0 to experiment the cnot effect. 

In [ ]:
# define registers, size 2
qr = QuantumRegister(2)
cr = ClassicalRegister(2)

In [ ]:
# define circuit
# add x on qubit0 (or not)
qc_cnot = QuantumCircuit(qr,cr)


qc_cnot.x(qr[0])

# add cnot controlled bit qbit0, target qubit1

qc_cnot.cx(qr[0],qr[1])
# add measure

qc_cnot.measure(qr,cr)

# add draw
qc_cnot.draw()

In [ ]:
# execute, get results 

job = execute(qc_cnot,backend, shots=1024)
result = job.result()
result.get_counts(qc_cnot)

In [ ]:
# change the value in get_count method (your circuit name)
plot_histogram(result.get_counts(qc_cnot))

## <span style="color:red"><em>Example 4: CONTROL-CONTROL-NOT Toffoli gate</em></span>

if a = 1 and b = 1, then flip c

Note: <span style="color:green"><em>Toffoli gate is universal</em></span>

control-control-not is : 

`<circuit>.ccnot(control_qubitA, control_qubitB, taget_qubit)`

In [ ]:
# define registers, size 3
qr = QuantumRegister(3)

cr = ClassicalRegister(3)

In [ ]:
# define quantum circuit

qc_ccnot = QuantumCircuit(qr,cr)

# add x on qubit0 and: or qubit 1 to view different results 

qc_ccnot.x(qr[0])

# add ccnot, measure, draw 

qc_ccnot.ccx(qr[0],qr[1],qr[2])
qc_ccnot.measure(qr,cr)
qc_ccnot.draw()

In [ ]:
# execute, get results, plot
job = execute(qc_ccnot,backend, shots=1024)
result = job.result()
result.get_counts(qc_ccnot)
plot_histogram(result.get_counts(qc_ccnot))

## <span style="color:red"><em>Example 5: CONTROL-SWAP : Fredkin gate</em></span>

if a = 1 then swap b and c states.

Note: Quantum gates are reversible. 


control-swap is : 

`<circuit>.cswap(control_qubit, swapped_qubit, swapped_qubit)`

In [ ]:
# define registers, circuit, add X on qubit 0, and/or 1, and/or 2 to view differentt cases: 

qc_cswap = QuantumCircuit(qr,cr)
qc_cswap.x(qr[0])
qc_cswap.x(qr[1])

#add cswap

qc_cswap.cswap(qr[0],qr[1],qr[2])

#add measure and draw

qc_cswap.measure(qr,cr)
qc_cswap.draw()

In [ ]:
# execute, get results, plot...

job = execute(qc_cswap,backend, shots=1024)
result = job.result()
result.get_counts(qc_cswap)
plot_histogram(result.get_counts(qc_cswap))

## <span style="color:red"><em>Example 6: What does this do ?</em></span>

This is a basic 2 qubits gate, can you figure out what this does ? 

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
mystery = QuantumCircuit(qr,cr)
###########################
mystery.cx(qr[0],qr[1])
mystery.cx(qr[1],qr[0])
mystery.cx(qr[0],qr[1])
mystery.measure(qr,cr)
###########################
mystery.draw()

.... Hint, try this (will run the above circuit against all input combinations 00,01,10,11 and will show you the result): 

In [ ]:
print("       q0 q1")
print("       -- --")
for i in range(4): 
    qc = QuantumCircuit(qr,cr)
    if i%2: 
        qc.x(qr[0])
    if i>1:
        qc.x(qr[1])
    # this is the same circuit as above #
    qc.cx(qr[0],qr[1])
    qc.cx(qr[1],qr[0])
    qc.cx(qr[0],qr[1])
    qc.measure(qr,cr)
    #####################################
    print(f"input:  {(i>1)*1} {i%2}")
    job = execute(qc,backend, shots=1024)
    result = job.result()
    for x in (result.get_counts(qc)):
        print(f"output: {x[0]} {x[1]}")
    print(" ")

# D. Some Quantum computing

##  <span style="color:red"><em>Single qbit : "superpositon"</em></span>

Let's do it again : define registers (size 1), quantum circuit, add `h` gate, and `measure` gate, draw circuit. 

In [ ]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
qc_sup = QuantumCircuit(qr,cr)
qc_sup.h(qr[0])
qc_sup.measure(qr,cr)
qc_sup.draw()

execute and get results

In [ ]:
job = execute(qc_sup,backend, shots=8096)
result = job.result()
result.get_counts(qc_sup)
plot_histogram(result.get_counts(qc_sup))

<img src="./images/h_bloch2.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="350 px" align="left">

### See this page for more on what happens on the "Bloch Sphere" when using this and other quantum gates : https://javafxpert.github.io/grok-bloch/

#### More on moving in the Bloch-Sphere ?

In [ ]:
from math import sqrt
from qiskit import QuantumCircuit, QuantumRegister, BasicAer, execute


## Setting up:
#
num_qubits = 1
qr = QuantumRegister(num_qubits, "qr")
circ = QuantumCircuit(qr)


## Building the circuit:
#
circ.x( qr[0] )     # inverting the first and only qubit

## Printing the circuit:
#
print( circ.draw().single_string() )
print(" ")

## Setting up the statevector simulator:
#
simulator = BasicAer.get_backend('statevector_simulator')

## Running the simulator with the circuit on different input states:
#
input_state_zero    = [1, 0]                        # state |0>     or Z+
input_state_one     = [0, 1]                        # state |1>     or Z-
input_state_plus    = [1/sqrt(2),   1/sqrt(2) ]     # state |+>     or X+
input_state_minus   = [1/sqrt(2),  -1/sqrt(2) ]     # state |->     or X-
input_state_eye     = [1/sqrt(2),  1j/sqrt(2) ]     # state |i>     or Y+
input_state_mye     = [1/sqrt(2), -1j/sqrt(2) ]     # state |-i>    or Y-

for input_statevector in (input_state_zero, input_state_one, input_state_plus, input_state_minus, input_state_eye, input_state_mye):

    print( "Input statevector : {}".format(input_statevector) )
    output_statevector = list( execute(circ, simulator, backend_options={"initial_statevector": input_statevector}).result().get_statevector(circ) )
    print( "Output statevector: {}".format(output_statevector) )
    print("")


##  <span style="color:red"><em>What is superposition exactly ?</em></span>

Now try to build a quantum circuit, using a single qubit, use H then H then measure, run ..

In [ ]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
qc_sup = QuantumCircuit(qr,cr)
qc_sup.h(qr[0])
qc_sup.h(qr[0])
qc_sup.measure(qr,cr)
qc_sup.draw()

In [ ]:
job = execute(qc_sup,backend, shots=1024)
result = job.result()
result.get_counts(qc_sup)
plot_histogram(result.get_counts(qc_sup))

### Here is what happened ! (hope this helps :-) )
<br>
Let's define states |0⟩, |1⟩ as unitary basis vectors in a two dimensional space, we can write them in this form: 

\begin{equation}
|0⟩ = 
\left(
\begin{array}{cc}
 1 \\
 0 \\
\end{array}
\right)
\hspace{1cm}
|1⟩ = 
\left(
\begin{array}{cc}
 0 \\
 1 \\
\end{array}
\right)
\hspace{1cm}
\end{equation}

Then we define H (Hadamard operator) with this matrix : 


\begin{equation}
H = \frac{1}{\sqrt{2}}
\left(
\begin{array}{cc}
 1 & 1  \\
 1 & -1  \\
\end{array}
\right)
\hspace{1cm}
\end{equation}


It is easy to verify that : 


\begin{equation}
H|0⟩ = \frac{1}{\sqrt{2}} 
\left(
\begin{array}{cc}
 1 \\
 1 \\
\end{array}
\right)
\end{equation}

also, if we multiply this result again by H, we will find : 
\begin{equation}
HH|0⟩ =
\left(
\begin{array}{cc}
 1 \\
 0 \\
\end{array}
\right)
\end{equation}

That is :

\begin{equation} 
HH|0⟩ = |0⟩
\end{equation}

Which makes sense because it is also easy to verify that : 

\begin{equation} 
H^2 = I
\end{equation}

where:

\begin{equation}
I = 
\left(
\begin{array}{cc}
 1 & 0 \\
 0 & 1 \\
\end{array}
\right)
\end{equation}

##  <span style="color:red"><em>Multi qbit : entanglement</em></span>

Now build a quantum circuit using 2 qubits, start with H on qbit0 and then CNOT controlled by qubit0 targetting qbit1: 

`<cicruit>.cx(qr[0],qr[1])`

In [ ]:
#define registers, define curcuit, build circuit, draw.
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc_bell = QuantumCircuit(qr,cr)
qc_bell.h(qr[0])
qc_bell.cx(qr[0],qr[1])
qc_bell.measure(qr,cr)
qc_bell.draw()

In [ ]:
# define, execute job and plot results
job = execute(qc_bell,backend, shots=1024)
result = job.result()
result.get_counts(qc_bell)
plot_histogram(result.get_counts(qc_bell))

### What does this mean ? ... 

This is called a Bell state and can be written as : 

\begin{equation} 
|Bell⟩ = \frac{1}{\sqrt{2}}\left(|00⟩ + |11⟩ \right) 
\end{equation}

Let's assume we can factor it (find two states the product of which is the Bell state), for example: 

\begin{equation} 
|\phi⟩ = a|0⟩ + (b+ic)|1⟩  
\end{equation}
and 
\begin{equation} 
|\psi⟩ = d|0⟩ + (e+if)|1⟩  
\end{equation}

Then: 
\begin{equation} 
|\phi⟩|\psi⟩ = (ad|00⟩ + (ae + iaf)|01⟩ + (db+idc)|10⟩ + (b+ic)(e+if)|11⟩)  
\end{equation}

As |00⟩, |01⟩, |10⟩, |11⟩ are basis vectors, we can identify:  

\begin{equation}
ad = \frac{1}{\sqrt{2}}
\end{equation}
\begin{equation}
ae + iaf = 0
\end{equation}
\begin{equation}
db + idc = 0
\end{equation}
\begin{equation}
be - cf + i(bf+ce) = \frac{1}{\sqrt{2}}
\end{equation}

then a and d are both not null, so e = f = b = c = 0, with that: be - cf = 0, but this does not work with : 
\begin{equation}
be - cf = \frac{1}{\sqrt{2}}
\end{equation}

Proving that the Bell state cannot be written as a product state. 


### ... it means we cannot know anything about the separate components of the entangled system, we can only know about the whole.

##  <span style="color:red"><em>Bell state : run on actual quantum hardware</em></span>


In [ ]:
# sign on to IBM Q Experience, get your API Token, write it in a file (eg Qconfig.py), in a line:  
# APItoken = '<your_API_token'>

import Qconfig

#  now I can use Qconfig.APItoken to get the API key value 

In [ ]:
from qiskit import IBMQ
IBMQ.enable_account(Qconfig.APItoken)

In [ ]:
#my_back_end = IBMQ.get_backend('ibmq_5_tenerife')
my_back_end = IBMQ.get_backend('ibmq_16_melbourne')

In [ ]:
print(my_back_end.status())

In [ ]:
job = execute(qc_bell,my_back_end, shots=1024)
result = job.result()
result.get_counts(qc_bell)
plot_histogram(result.get_counts(qc_bell))

##  <span style="color:red"><em>Bernstein-Vazirani : "oracle"</em></span>
### towards computing

In [ ]:
q = QuantumRegister(8)
tmp = QuantumRegister(1)
res = ClassicalRegister(8)

In [ ]:
# secret = 1101
s = 13
oracle = QuantumCircuit(q,tmp,res)

In [ ]:
for i in range(len(q)):
    if ( s & (1 << i) ):
        oracle.cx(q[i],tmp[0])

In [ ]:
bv = QuantumCircuit(q,tmp,res)
bv.x(tmp[0])
bv.h(q)
bv.h(tmp)
bv += oracle
bv.h(q)
bv.h(tmp)
bv.measure(q,res)

In [ ]:
bv.draw()

In [ ]:
my_job = execute([bv], backend, shots=100)

In [ ]:
result = my_job.result()
result.get_counts(bv)
print(result.get_counts(bv))
from qiskit.tools.visualization import plot_histogram
plot_histogram (result.get_counts(bv))

# E. Further readings

## <a href="https://www.research.ibm.com/ibm-q/">IBM Q informations, resources</a>

<img src="./images/IBMQ.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="left">


### <a href="https://qiskit.org/">QISKit : documentations, tutorials, GitHub</a>

<img src="./images/qiskitOrg.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="left">



## <a href="https://www.youtube.com/channel/UClBNq7mCMf5xm8baE_VMl3A">QISKit channel on Youtube</a>
<img src="./images/qiskitYoutube.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="left">


### <a href="https://fr.wikipedia.org/wiki/Calculateur_quantique">Wikipedia: Calculateur Quantique</a>
<img src="./images/QCWikipedia.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="left">

## Isaac Chuang & Peter Shor, Quantum Information Science I, Part 1
### Edx MOOC
<br>
<img src="./images/MOOCedX.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="center">
<br>

## Leonard Susskind, Art Freidman: Mécanique Quantique, Le minimum Théorique
### Presses polytechniques et universitaires romandes
<br>
<img src="./images/susskindFriedman.jpg" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center">
<br>

## David McMahon, Quantum Computing Explained 
### Wiley
<br>
<img src="./images/mcmahon.jpg" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center">
<br>

### David Mermin, Calculs et Algorithmes Quantiques, 
#### EDP Sciences, CNRS Editions
<br>
<img src="./images/mermin.jpg" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center">
<br>

## Charles Corge, L'informatique Quantique, qu'est-ce et pour quoi faire ? 
### Ellipses
<br>
<img src="./images/corge.jpeg" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center">
<br>

## Michael A. Nielsen and Isaac L. Chuang : Quantum Computation and Quantum Information
### Cambridge
<br>
<img src="./images/nielsenChuang.jpg" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="400 px" align="center">
<br>